# Using AI21 Summarize on SageMaker through Model Packages

This sample notebook shows you how to deploy **AI21 Summarize** using Amazon SageMaker.

## Pre-requisites:
1. Before running this notebook, please make sure you got this notebook from the model catalog on SageMaker AWS Management Console.
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**.
1. This notebook is intended to work with **boto3 v1.25.4** or higher.

## Contents:
1. [Select model package](#1.-Select-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Summarize a single paragraph](#B.-Summarize-a-single-paragraph)
   3. [Summarize longer texts](#C.-Summarize-longer-texts)
   4. [Delete the endpoint](#D.-Delete-the-endpoint)
3. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Select model package
Confirm that you received this notebook from the model catalog in SageMaker AWS Management Console.

In [2]:
model_package_map = {
    "us-east-1": "arn:aws:sagemaker:us-east-1:865070037744:model-package/summarize-1-1-000-952cc95c07d034cfbae0477f25f26df0",
    "us-east-2": "arn:aws:sagemaker:us-east-2:057799348421:model-package/summarize-1-1-000-952cc95c07d034cfbae0477f25f26df0",
    "us-west-1": "arn:aws:sagemaker:us-west-1:382657785993:model-package/summarize-1-1-000-952cc95c07d034cfbae0477f25f26df0",
    "us-west-2": "arn:aws:sagemaker:us-west-2:594846645681:model-package/summarize-1-1-000-952cc95c07d034cfbae0477f25f26df0",
    "ca-central-1": "arn:aws:sagemaker:ca-central-1:470592106596:model-package/summarize-1-1-000-952cc95c07d034cfbae0477f25f26df0",
    "eu-central-1": "arn:aws:sagemaker:eu-central-1:446921602837:model-package/summarize-1-1-000-952cc95c07d034cfbae0477f25f26df0",
    "eu-west-1": "arn:aws:sagemaker:eu-west-1:985815980388:model-package/summarize-1-1-000-952cc95c07d034cfbae0477f25f26df0",
    "eu-west-2": "arn:aws:sagemaker:eu-west-2:856760150666:model-package/summarize-1-1-000-952cc95c07d034cfbae0477f25f26df0",
    "eu-west-3": "arn:aws:sagemaker:eu-west-3:843114510376:model-package/summarize-1-1-000-952cc95c07d034cfbae0477f25f26df0",
    "eu-north-1": "arn:aws:sagemaker:eu-north-1:136758871317:model-package/summarize-1-1-000-952cc95c07d034cfbae0477f25f26df0",
    "ap-southeast-1": "arn:aws:sagemaker:ap-southeast-1:192199979996:model-package/summarize-1-1-000-952cc95c07d034cfbae0477f25f26df0",
    "ap-southeast-2": "arn:aws:sagemaker:ap-southeast-2:666831318237:model-package/summarize-1-1-000-952cc95c07d034cfbae0477f25f26df0",
    "ap-northeast-2": "arn:aws:sagemaker:ap-northeast-2:745090734665:model-package/summarize-1-1-000-952cc95c07d034cfbae0477f25f26df0",
    "ap-northeast-1": "arn:aws:sagemaker:ap-northeast-1:977537786026:model-package/summarize-1-1-000-952cc95c07d034cfbae0477f25f26df0",
    "ap-south-1": "arn:aws:sagemaker:ap-south-1:077584701553:model-package/summarize-1-1-000-952cc95c07d034cfbae0477f25f26df0",
    "sa-east-1": "arn:aws:sagemaker:sa-east-1:270155090741:model-package/summarize-1-1-000-952cc95c07d034cfbae0477f25f26df0"
}

In [3]:
import json
from sagemaker import ModelPackage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
import sagemaker as sage
import boto3

In [4]:
# Import wikipedia for longer texts example
! pip install wikipedia
import wikipedia

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


### Check the version of boto3 - must be v1.25.4 or higher
If you see a lower version number, pick another kernel to run the notebook, with Python 3.8 or above

In [5]:
boto3.__version__

'1.26.74'

### Install ai21 python SDK

In [6]:
! pip install -U "ai21[SM]"
import ai21

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [7]:
region = boto3.Session().region_name
if region not in model_package_map.keys():
    raise ("UNSUPPORTED REGION")

model_package_arn = model_package_map[region]

In [8]:
role = get_execution_role()
sagemaker_session = sage.Session()

runtime_sm_client = boto3.client("runtime.sagemaker")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html).

In [9]:
model_name = "summarize"

content_type = "application/json"

real_time_inference_instance_type = (
    "ml.g4dn.12xlarge"
)

### A. Create an endpoint

In [12]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name, 
                         model_data_download_timeout=3600,
                         container_startup_health_check_timeout=600,
                        )

---------!

Once endpoint has been created, you would be able to perform real-time inference.

### B. Summarize a single paragraph

**AI21 Summarize model** offers access to our world-class summarization engine. It has been specifically developed for understanding long texts and providing a faithful summary of the original document.

This model takes a piece of text and generates summaries that remain faithful to the original document (i.e. no external information is added during the process).

The input text should contain **at least 40 words** and **no more than 50,000 characters**.

The following is an example of a summary of a single paragraph taken from a news article.

In [13]:
text = """The error affected a number of international flights leaving the terminal on Wednesday, with some airlines urging passengers to travel only with hand luggage.
Virgin Atlantic said all airlines flying out of the terminal had been affected.
Passengers have been warned it may be days before they are reunited with luggage.
An airport spokesperson apologised and said the fault had now been fixed.
Virgin Atlantic said it would ensure all bags were sent out as soon as possible.
It added customers should retain receipts for anything they had bought and make a claim to be reimbursed.
Passengers, who were informed by e-mail of the problem, took to social media to vent their frustrations.
One branded the situation "ludicrous" and said he was only told 12 hours before his flight.
The airport said it could not confirm what the problem was, what had caused it or how many people had been affected."""

response = ai21.Summarize.execute(
                          source=text,
                          sourceType="TEXT",
                          sm_endpoint="summarize"
)

print("Original text:")
print(text)
print("================")
print("Summary:")
print(response.summary)

Original text:
The error affected a number of international flights leaving the terminal on Wednesday, with some airlines urging passengers to travel only with hand luggage.
Virgin Atlantic said all airlines flying out of the terminal had been affected.
Passengers have been warned it may be days before they are reunited with luggage.
An airport spokesperson apologised and said the fault had now been fixed.
Virgin Atlantic said it would ensure all bags were sent out as soon as possible.
It added customers should retain receipts for anything they had bought and make a claim to be reimbursed.
Passengers, who were informed by e-mail of the problem, took to social media to vent their frustrations.
One branded the situation "ludicrous" and said he was only told 12 hours before his flight.
The airport said it could not confirm what the problem was, what had caused it or how many people had been affected.
Summary:
The error affected a number of international flights leaving the terminal on Wed

### C. Summarize longer texts

AI21 Summarize can handle inputs up to 50,000 characters, as stated above. This translates into roughly 10,000 words, or a whopping 40 pages.

As a demonstration of the model's behavior, we'll load a page from Wikipedia. If you're an Office fan, this will look familiar :)

In [14]:
wiki_page = wikipedia.page("The_Office_(American_TV_series)").content

Since this is a full page, we will focus only the opening section.

In [15]:
office_text = wiki_page.split("\n\n")[0]
print(office_text)

The Office is an American mockumentary sitcom television series that depicts the everyday work lives of office employees at the Scranton, Pennsylvania, branch of the fictional Dunder Mifflin Paper Company. It aired on NBC from March 24, 2005, to May 16, 2013, spanning a total of nine seasons. Based on the 2001–2003 BBC series of the same name created by Ricky Gervais and Stephen Merchant, it was adapted for American television by Greg Daniels, a veteran writer for Saturday Night Live, King of the Hill, and The Simpsons. It was co-produced by Daniels' Deedle-Dee Productions and Reveille Productions (later Shine America), in association with Universal Television. The original executive producers were Daniels, Gervais, Merchant, Howard Klein, and Ben Silverman, with numerous others being promoted in later seasons.
Like its British counterpart, the series was filmed in a single-camera setup without a studio audience or a laugh track to simulate the look of an actual documentary. The series

The summaries are formatted as bullet lists, following the original text flow.

In [16]:
response = ai21.Summarize.execute(
                          source=office_text,
                          sourceType="TEXT",
                          sm_endpoint="summarize"
)

print(response.summary)

The Office is an American mockumentary sitcom television series based on the 2001 - 2003 BBC series of the same name created by Ricky Gervais and Stephen Merchant. It aired on NBC from March 24, 2005, to May 16, 2013, spanning a total of nine seasons.
The Office debuted on NBC as a mid-season replacement and aired 201 episodes. It featured Steve Carell, Rainn Wilson, John Krasinski, Jenna Fischer, and B. J. Novak.
The Office was met with mixed reviews during its short first season, but received significant acclaim from television critics in the following seasons.
The eighth season was criticized for a decline in quality, but the ninth and final season ended the series with a generally positive response.


Here is the full text and the bullet points summarized:

In [17]:
print("Original text:")
print(office_text)
print("====================")
print("Summary:")
summaries = response.summary.split("\n")
for summary in summaries:
    print("- " + summary)

Original text:
The Office is an American mockumentary sitcom television series that depicts the everyday work lives of office employees at the Scranton, Pennsylvania, branch of the fictional Dunder Mifflin Paper Company. It aired on NBC from March 24, 2005, to May 16, 2013, spanning a total of nine seasons. Based on the 2001–2003 BBC series of the same name created by Ricky Gervais and Stephen Merchant, it was adapted for American television by Greg Daniels, a veteran writer for Saturday Night Live, King of the Hill, and The Simpsons. It was co-produced by Daniels' Deedle-Dee Productions and Reveille Productions (later Shine America), in association with Universal Television. The original executive producers were Daniels, Gervais, Merchant, Howard Klein, and Ben Silverman, with numerous others being promoted in later seasons.
Like its British counterpart, the series was filmed in a single-camera setup without a studio audience or a laugh track to simulate the look of an actual document

### Interested in learning more?
Take a look at our [guide](https://docs.ai21.com/docs/summarize-api) to understand all the capabilities of AI21 Summarize model

### D. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [18]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 4. Clean-up

### A. Delete the model

In [19]:
model.delete_model()